In [1]:
import pandas as pd
import os
import numpy as np
from collections import defaultdict

# 输入
csv_dir = "../summary/zhang"  # 标注结果所在文件夹（会寻找子目录），后续处理所有`.csv`文件
video_dir = "../target/step1/data"  # 标注视频所在文件夹（会寻找子目录），后续处理所有`.mp4`文件

# 一些参数
avaiable_cameras = ["2m", "3m", "4m"]  # 处理指定摄像头的数据，若为空则处理所有
# avaiable_person_ids = [130]
avaiable_person_ids = range(1, 13)
avaiable_persons = [
    "P%04d" % i for i in avaiable_person_ids
]  # 处理指定人物数据，若为空则处理所有

# 导出标记错误
enable_labeling_err = True
labeling_err_file = "./labeling_error.txt"

# 导出统计结果
enable_summary = True
summary_file = "./summary.csv"

# 导出质量管理反馈结果
enable_quality_feedback = True
qualify_feedback_file = "./quality_feedback.txt"

# 所有labels的集合
frame_type = {"medium", "end"}
pose_type = {"stand", "sit", "squat", "lie", "half_lie"}
actions = [
    "close",
    "drinking",
    "eating",
    "knock",
    "medicine",
    "open",
    "pushoverfall",
    "stillfall",
    "takecup",
    "takephone",
    "tripfall",
    "walkingfall",
]
action_type = set(actions)
quality_type = {
    "qualified",
    "err_camera_lacking",
    "err_action_lacking",
    "err_light",
    "err_pose",
    "err_sleeve",
    "err_view",
    "err_shelter",
    "err_unknown",
}
total_type = frame_type | pose_type | action_type | quality_type

series_error_type = {"err_camera_lacking", "err_action_lacking", "err_unknown"}
remade_error_type = {"err_action_lacking"}
useless_error_type = {"err_camera_lacking", "err_unknown"}
typo_error_type = {
    "err_light",
    "err_pose",
    "err_sleeve",
    "err_view",
    "err_shelter",
}


In [2]:
# 中英文转换相关
translations_dict = {
    "qualified": "合格",
    "err_camera_lacking": "画面缺失",
    "err_action_lacking": "动作不完整或动作标注错误",
    "err_light": "光照与文件名不符",
    "err_pose": "姿态与文件名不符",
    "err_sleeve": "衣着与文件名不符",
    "err_view": "视角与文件名不符",
    "err_shelter": "遮挡与文件名不符",
    "err_unknown": "未知错误",
    "medium": "中间帧",
    "end": "结尾帧",
    "stand": "站",
    "sit": "坐",
    "squat": "蹲",
    "lie": "躺",
    "half_lie": "过渡",
    "stillfall": "原地软倒",
    "walkingfall": "行进软倒",
    "pushoverfall": "推倒",
    "tripfall": "绊倒",
    "takephone": "拿手机",
    "takecup": "拿水杯",
    "eating": "吃饭",
    "drinking": "喝水",
    "medicine": "吃药",
    "knock": "磕碰",
    "close": "关门",
    "open": "开门",
    "Not_Labeled_or_Wrong_Labeled": "未标注或标注无效",
    "No_Data": "无数据",
}


def _trans_func(context):
    """
    将英文结果转换为中文
    """
    if len(context) > 0:
        segs = set(context.split(" "))
        res = []
        for seg in segs:
            if seg == "":
                continue
            if seg not in translations_dict:
                print("unknown english `{}`".format(seg))
            else:
                res.append(translations_dict[seg])
        context = " ".join(res)
    return context

In [3]:
# 获取所有视频
video_list = []
for cur_path, _, file_names in os.walk(video_dir):
    for file_name in file_names:
        if file_name.endswith(".mp4"):
            row = file_name.split("_")
            if (
                row[1] in avaiable_cameras
                and row[-1][:5] in avaiable_persons
            ):
                video_list.append(file_name)

# 获取视频 dict
# key: {pid}_{camera}
# value: list(actions)
total_samples = defaultdict(list)
for file_name in video_list:
    row = file_name.split("_")
    action = row[0]
    camera = row[1]
    person = row[-1][:5]
    key = "{}_{}".format(person, camera)
    total_samples[key].append(action)

In [4]:
def _get_and_concat_all_csvs(csv_dir):
    """获取并拼接所有 csv 结果"""
    csv_list = []
    for cur_path, _, file_names in os.walk(csv_dir):
        csv_list += [
            os.path.join(cur_path, file_name)
            for file_name in file_names
            if file_name.endswith(".csv")
        ]
    return pd.concat([pd.read_csv(csv_file) for csv_file in csv_list])

def _filter_df_by_camera_and_person(df, avaiable_cameras, avaiable_persons):
    """
    根据 camera 编号和 person 编号筛选数据
    """
    df["camera"] = df["image"].str.split("_").map(lambda x: x[1])
    df["person"] = df["image"].str.split("_").map(lambda x: x[-1][:5])
    if len(avaiable_cameras) > 0:
        df = df[df["camera"].isin(avaiable_cameras)]
    if len(avaiable_persons) > 0:
        df = df[df["person"].isin(avaiable_persons)]
    return df


# 获取原始df，并根据 camera 和 person 筛选数据
df = _get_and_concat_all_csvs(csv_dir)
df = _filter_df_by_camera_and_person(df, avaiable_cameras, avaiable_persons)
df['sample'] = df['image'].str.split(".").apply(lambda x: x[0])
df.head()

,image,xmin,ymin,xmax,ymax,label,camera,person,sample
0,close_2m_stand_coldlight_shortsleeve_frontshot...,1144.589938,328.235294,1340.427292,1080.0,stand,2m,P0007,close_2m_stand_coldlight_shortsleeve_frontshot...
1,close_2m_stand_coldlight_shortsleeve_frontshot...,956.691937,340.147059,1261.724643,1080.0,stand,2m,P0007,close_2m_stand_coldlight_shortsleeve_frontshot...
2,close_2m_stand_coldlight_shortsleeve_frontshot...,956.691937,340.147059,1261.724643,1080.0,medium,2m,P0007,close_2m_stand_coldlight_shortsleeve_frontshot...
3,close_2m_stand_coldlight_shortsleeve_frontshot...,956.691937,340.147059,1261.724643,1080.0,qualified,2m,P0007,close_2m_stand_coldlight_shortsleeve_frontshot...
4,close_2m_stand_coldlight_shortsleeve_frontshot...,956.691937,340.147059,1261.724643,1080.0,close,2m,P0007,close_2m_stand_coldlight_shortsleeve_frontshot...


In [5]:
def _quality_group_by_sample(x):
    labels = list(x['label'].unique())
    cur_labels = " ".join(labels)
    row = x.iloc[0, 0].split("_")
    pid_camera = "{}_{}".format(row[-1][:5], row[1])
    action = row[0]

    labeling_err = False
    if len(labels) > 1 and 'qualified' in labels:
        # 质量管理标签标记错误只有一种情况，同时存在“合格”与“不合格”标签
        labeling_err = True
    
    most_series_err = ""
    most_series_err_type = ""
    cur_labels_set = set(labels)
    if len(cur_labels_set & useless_error_type) > 0:
        most_series_err = " ".join(list(cur_labels_set & useless_error_type))
        most_series_err_type = "useless"
    elif len(cur_labels_set & remade_error_type) > 0:
        most_series_err = " ".join(list(cur_labels_set & remade_error_type))
        most_series_err_type = "remade"
    elif len(cur_labels_set & typo_error_type) > 0:
        most_series_err = " ".join(list(cur_labels_set & typo_error_type))
        most_series_err_type = "typo"
    
    return pd.Series([pid_camera, action, cur_labels, most_series_err, most_series_err_type, labeling_err])


def _quality_summary_groupby(x):
    index_name = x['index_name'].unique()[0]
    res = [''] * len(actions)
    for row in x.iterrows():
        res[actions.index(row[1]['action'])] = row[1]['quality_labels']
    for i in range(len(res)):
        if len(res[i]) == 0:
            res[i] = 'No_Data' if actions[i] not in total_samples[index_name] else 'Not_Labeled'
    return pd.Series(res)


# 获取质量相关数据
quality_df = df[df["label"].isin(quality_type)]
quality_df = quality_df.groupby('sample').apply(_quality_group_by_sample)
quality_df.columns = ["index_name", "action", "quality_labels", "most_series_error", "most_series_error_type", "labeling_error"]

# TODO: 导出标记错误
# 这里的标记错误，指的是同时存在“合格”与“不合格”质量管理标签。

# 获取质量统计结果
quality_summary_df = quality_df.groupby('index_name').apply(_quality_summary_groupby)
quality_summary_df.columns = actions
quality_summary_df.head()

,close,drinking,eating,knock,medicine,open,pushoverfall,stillfall,takecup,takephone,tripfall,walkingfall
index_name,,,,,,,,,,,,
P0007_2m,qualified,qualified,qualified,qualified,qualified,qualified,qualified,qualified,qualified,qualified,qualified,qualified
P0007_3m,qualified,qualified,qualified,qualified,qualified,qualified,qualified,qualified,qualified,qualified,qualified,qualified
P0007_4m,qualified,qualified,qualified,qualified,qualified,qualified,qualified,qualified,qualified,qualified,qualified,qualified
P0008_2m,qualified,qualified,qualified,qualified,qualified,qualified,err_action_lacking,qualified,qualified,qualified,err_action_lacking,qualified
P0008_3m,qualified,qualified,qualified,qualified,qualified,qualified,err_action_lacking,err_action_lacking,qualified,err_action_lacking,err_action_lacking,qualified


In [6]:
# 中文版质量管理 summary
quality_summary_ch_df = quality_summary_df.copy()
for col in quality_summary_ch_df.columns:
    quality_summary_ch_df[col] = quality_summary_ch_df[col].apply(_trans_func)
quality_summary_ch_df.columns = [
    translations_dict[k] for k in list(quality_summary_ch_df.columns)
]
quality_summary_ch_df.head()

,关门,喝水,吃饭,磕碰,吃药,开门,推倒,原地软倒,拿水杯,拿手机,绊倒,行进软倒
index_name,,,,,,,,,,,,
P0007_2m,合格,合格,合格,合格,合格,合格,合格,合格,合格,合格,合格,合格
P0007_3m,合格,合格,合格,合格,合格,合格,合格,合格,合格,合格,合格,合格
P0007_4m,合格,合格,合格,合格,合格,合格,合格,合格,合格,合格,合格,合格
P0008_2m,合格,合格,合格,合格,合格,合格,动作不完整或动作标注错误,合格,合格,合格,动作不完整或动作标注错误,合格
P0008_3m,合格,合格,合格,合格,合格,合格,动作不完整或动作标注错误,动作不完整或动作标注错误,合格,动作不完整或动作标注错误,动作不完整或动作标注错误,合格


In [7]:
# 导出质量管理反馈结果

def _outputs_quality_feedback(qualify_feedback_file, summary_df):
    """
    根据标记汇总结果，输出质量管理反馈信息
    """
    qualify_feedback_writer = open(qualify_feedback_file, "w")
    remade_list = []
    useless_list = []
    typo_list = []

    # 遍历标记汇总结果，保存所有质量问题
    # 并根据质量问题的严重程度进行分类
    for row in summary_df.iterrows():
        index_name = row[0]
        for action in row[1].index:
            qualities = set(row[1][action].split(" "))
            context = " ".join(
                [
                    index_name,
                    translations_dict[action],
                    _trans_func(row[1][action]),
                ]
            )
            if len(qualities & useless_error_type) > 0:
                useless_list.append(context)
            if len(qualities & remade_error_type) > 0:
                remade_list.append(context)
            if len(qualities & typo_error_type) > 0:
                typo_list.append(context)

    # 按严重程度分别输出相关质量问题
    # 1. 无法使用的数据
    print("\n")
    context = "无效数据有 {} 个，分别是：".format(len(useless_list))
    print(context)
    qualify_feedback_writer.write(context + "\n")
    for context in useless_list:
        print(context)
        qualify_feedback_writer.write(context + "\n")
    print()
    qualify_feedback_writer.write("\n")

    # 2. 需要返工的数据
    context = "需要返工数据有 {} 个，分别是：".format(len(remade_list))
    print(context)
    qualify_feedback_writer.write(context + "\n")
    for context in remade_list:
        print(context)
        qualify_feedback_writer.write(context + "\n")
    print()
    qualify_feedback_writer.write("\n")

    # 有小问题，但不影响使用的数据
    context = "有小问题但不影响使用的数据有 {} 个，分别是：".format(len(typo_list))
    print(context)
    qualify_feedback_writer.write(context + "\n")
    for context in typo_list:
        print(context)
        qualify_feedback_writer.write(context + "\n")

    qualify_feedback_writer.close()

# 获取质量管理反馈结果
if enable_quality_feedback:
    _outputs_quality_feedback(qualify_feedback_file, quality_summary_df)



无效数据有 7 个，分别是：
P0009_2m 喝水 画面缺失
P0009_2m 磕碰 画面缺失
P0009_2m 吃药 画面缺失
P0009_2m 拿水杯 画面缺失
P0009_2m 拿手机 画面缺失
P0009_3m 磕碰 未知错误
P0009_4m 磕碰 未知错误

需要返工数据有 13 个，分别是：
P0008_2m 推倒 动作不完整或动作标注错误
P0008_2m 绊倒 动作不完整或动作标注错误
P0008_3m 推倒 动作不完整或动作标注错误
P0008_3m 原地软倒 动作不完整或动作标注错误
P0008_3m 拿手机 动作不完整或动作标注错误
P0008_3m 绊倒 动作不完整或动作标注错误
P0008_4m 推倒 动作不完整或动作标注错误
P0008_4m 原地软倒 动作不完整或动作标注错误
P0008_4m 拿手机 动作不完整或动作标注错误
P0008_4m 绊倒 动作不完整或动作标注错误
P0009_2m 绊倒 动作不完整或动作标注错误
P0009_3m 绊倒 动作不完整或动作标注错误
P0009_4m 绊倒 动作不完整或动作标注错误

有小问题但不影响使用的数据有 1 个，分别是：
P0009_3m 吃药 姿态与文件名不符


In [8]:
# bbox 统计（包括中间帧bbox、结尾帧bbox、普通bbox）
# 要求：统计上述数据时，不考虑存在严重错误（需要返工或返工也无效的数据）的样本
# 注意事项：如果一个样本中存在严重错误，应该只有一个bbox。如果有多个bbox需要提醒标注者，放在 labeling error 中。
# 统计一个表，也是横坐标为 pid_camera，纵坐标是12类actions
# 元素值分为三种可能性：无数据（没有原始视频）、存在严重问题（需要返工或返工也无效）、存在标注问题、bbox统计
# 存在标注问题，主要是各类标签数量可能不符合要求，比如单个bbox的姿态标签数量不止一个、帧类别标签不止一个，存在帧类别标签的bbox不没有姿态标签等，单个样本可能存在多个中间帧或多个结尾帧
# 所谓bbox统计，指的是中间帧bbox数量、结尾帧bbox数量、其他bbox数量

In [9]:
def _sample_group_by_bbox(x):
    # 记录当前bbox类别（中间帧、结尾帧、其他帧）
    # 记录当前bbox标注的错误：
    # 姿态标签数量不为一（如果不存在严重质量问题，那么要求所有bbox都对应一个人，要求有一个对应的姿态）
    # 帧类别标签数量不为一（同时为中间帧、结尾帧是不允许的）
    labels_list = list(x['label'])
    labels_set = set(labels_list)
    medium = labels_list.count('medium')
    end = labels_list.count('end')
    other = 0
    labeling_err = []
    if medium == 0 and end == 0:
        other = 1

    if medium != 0 and end != 0:
        labeling_err.append("同一个bbox不能同时作为中间帧与结尾帧")
    
    if len(labels_set & pose_type) != 1:
        labeling_err.append("同一个bbox的姿态标签不为一")
    
    return pd.Series([medium, end, other, " ".join(labeling_err)])

def _group_by_sample(x):
    labels_list = list(x['label'])
    labels_set = set(labels_list)
    row = x.iloc[0, 0].split("_")
    pid_camera = "{}_{}".format(row[-1][:5], row[1])
    action = row[0]

    labeling_err_list = []
    res_list = [pid_camera, action]

    # 1. 处理质量管理标签
    # 筛选不存在严重问题的数据，记录质量管理标签中可能存在的标注错误
    if len(labels_set & series_error_type) != 0:
        res_list.append("SeriesErrorData {}".format(list(labels_set & series_error_type)))
        # TODO: 记录标注错误信息
        res_list.append(" ".join(labeling_err_list))
        return pd.Series(res_list)
    
    # 2. 统计帧类别标签数量，一个样本中间帧、结尾帧的数量可能大于1

    # 3. 根据bbox进行group by操作
    # 记录可能出现的标注错误：姿态标签数量不为一，帧类别标签数量不为一，有帧类别标签或质量管理标签、但没有姿态标签
    x['bbox'] = (
        x["xmin"].astype(str)
        + x["ymin"].astype(str)
        + x["xmax"].astype(str)
        + x["ymax"].astype(str)
    )
    bbox_df = x.groupby('bbox').apply(_sample_group_by_bbox)
    bbox_df.columns = ['medium', 'end', 'other', 'labeling_err_details']
    res_list.append("{}-{}-{}".format(bbox_df['medium'].sum(), bbox_df['end'].sum(), bbox_df['other'].sum()))
    
    for cur_labeling_err in bbox_df['labeling_err_details'].unique():
        if len(cur_labeling_err) > 0:
            labeling_err_list += cur_labeling_err.split(" ")
    
    res_list.append(" ".join(list(set(labeling_err_list))))
    return pd.Series(res_list)

sample_df = df.groupby('sample').apply(_group_by_sample)
sample_df.columns = ['index_name', 'action', 'data', 'labeling_err_details']
sample_df.head()

,index_name,action,data,labeling_err_details
sample,,,,
close_2m_stand_coldlight_shortsleeve_frontshot_withshelter_CJ01_P0007,P0007_2m,close,1-0-1,
close_2m_stand_coldlight_shortsleeve_frontshot_withshelter_CJ01_P0008,P0008_2m,close,1-0-1,
close_2m_stand_coldlight_shortsleeve_frontshot_withshelter_CJ01_P0009,P0009_2m,close,1-0-1,
close_3m_stand_coldlight_shortsleeve_sideshot_withoutshelter_CJ01_P0007,P0007_3m,close,1-0-2,
close_3m_stand_coldlight_shortsleeve_sideshot_withoutshelter_CJ01_P0008,P0008_3m,close,1-0-1,


In [10]:
def _sample_group_by_index_name(x):
    index_name = x['index_name'].unique()[0]
    res = [''] * len(actions)
    for row in x.iterrows():
        labeling_err_details = row[1]['labeling_err_details']
        if len(labeling_err_details) > 0:
            r = 'LabelingError: ' + labeling_err_details
        else:
            r = row[1]['data']
        res[actions.index(row[1]['action'])] = r
    for i in range(len(res)):
        if len(res[i]) == 0:
            res[i] = 'No_Data' if actions[i] not in total_samples[index_name] else 'Not_Labeled'
    return pd.Series(res)


sample_summary_df = sample_df.groupby('index_name').apply(_sample_group_by_index_name)
sample_summary_df.columns = actions
sample_summary_df.head()

,close,drinking,eating,knock,medicine,open,pushoverfall,stillfall,takecup,takephone,tripfall,walkingfall
index_name,,,,,,,,,,,,
P0007_2m,1-0-1,2-0-0,2-0-0,1-0-0,1-0-0,1-0-1,1-0-1,1-0-0,1-0-0,1-0-0,1-0-0,1-0-0
P0007_3m,1-0-2,2-0-0,2-0-0,1-0-0,1-0-0,1-0-1,1-0-1,1-0-0,1-0-0,1-0-0,1-0-1,1-0-0
P0007_4m,1-0-1,2-0-5,2-0-4,1-0-2,1-0-2,1-0-1,1-0-2,1-0-3,1-0-2,1-0-2,1-0-2,1-0-2
P0008_2m,1-0-1,2-0-0,2-0-0,1-0-0,1-0-0,1-0-1,SeriesErrorData ['err_action_lacking'],1-0-0,1-0-0,1-0-0,SeriesErrorData ['err_action_lacking'],1-0-0
P0008_3m,1-0-1,2-0-0,2-0-0,1-0-0,1-0-0,1-0-0,SeriesErrorData ['err_action_lacking'],SeriesErrorData ['err_action_lacking'],1-0-0,SeriesErrorData ['err_action_lacking'],SeriesErrorData ['err_action_lacking'],1-0-0


In [11]:
# TODO： 增加错误类型
# 需要返工的错误类型包括：行为不完整、视频中行为与视频名称行为不符
# 返工也无效的数据：画面不完整、演员演技浮夸，总而言之，是肉眼不能分辨的行为
# 小错误：命名错误

In [ ]:
# TODO: write labeling error to local and console